In [ ]:
# core
import os
import re
import warnings
warnings.filterwarnings('ignore')

# data + plotting
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import missingno as msno
from scipy.stats import zscore
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()
np.set_printoptions(threshold=np.inf)

# sklearn (preprocessing / pipeline / model selection / metrics)
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, PolynomialFeatures, PowerTransformer
from sklearn.model_selection import train_test_split, KFold, cross_val_score, GridSearchCV
from sklearn.metrics import mean_squared_error

# classical models (if you use them elsewhere)
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.ensemble import RandomForestRegressor

# gradient boosting / lightgbm / xgboost
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor

# utilities
import joblib   # optional: save/load pipeline

from lightgbm import LGBMRegressor
from sklearn.model_selection import RandomizedSearchCV


In [ ]:
# Set plot style
sns.set(style="whitegrid")
plt.rcParams['figure.figsize'] = (12, 6)


# 1️⃣ Load Data
print("Loading data...")
df = pd.read_csv('../data/train.csv')
df.columns = df.columns.str.strip()
display(df.head())
print(f"Initial data shape: {df.shape}")

# 2️⃣ Clean all string/object columns: strip spaces, replace blanks with NaN
print("Cleaning string columns...")
for col in df.select_dtypes(include='object').columns:
    df[col] = df[col].astype(str).str.strip()
    df[col] = df[col].replace({'': np.nan, 'nan': np.nan, 'NaN': np.nan})

# 3️⃣ Normalize Yes/No columns to consistent "Yes"/"No"
print("Normalizing Yes/No columns...")
yes_no_cols = ['CrossBorder_Shipping', 'Urgent_Shipping', 'Installation_Service',
               'Fragile_Equipment', 'Rural_Hospital']
for col in yes_no_cols:
    if col in df.columns:
        df[col] = df[col].replace({
            'YES': 'Yes', 'yes': 'Yes', 'Y': 'Yes', 'y': 'Yes',
            'NO': 'No', 'no': 'No', 'N': 'No', 'n': 'No'
        })

# 4️⃣ Convert date columns to datetime
print("Converting date columns...")
df['Order_Placed_Date'] = pd.to_datetime(df['Order_Placed_Date'], errors='coerce')
df['Delivery_Date'] = pd.to_datetime(df['Delivery_Date'], errors='coerce')

# 5️⃣ Create new feature: Delivery_Days (difference in days)
print("Engineering Delivery_Days feature...")
df['Delivery_Days'] = (df['Delivery_Date'] - df['Order_Placed_Date']).dt.days
df['Delivery_Days'] = pd.to_numeric(df['Delivery_Days'], errors='coerce')

# === ADDED: Date Feature Engineering ===
print("Engineering more date features...")
df['Order_Month'] = df['Order_Placed_Date'].dt.month
df['Order_Day_of_Week'] = df['Order_Placed_Date'].dt.dayofweek  # Monday=0, Sunday=6
df['Order_Is_Weekend'] = df['Order_Day_of_Week'].isin([5, 6])
# === END ADDED ===

# 6️⃣ (Original) delete initial date rows
# df = df.dropna(subset=['Order_Placed_Date', 'Delivery_Date'])

# 7️⃣ Drop exact duplicate rows
print("Dropping duplicates...")
before = len(df)
df = df.drop_duplicates()
after = len(df)
print(f"Dropped {before - after} duplicate rows.")

# 8️⃣ Quick check after cleaning
print("\n" + "="*30)
print(" CLEANING & FEATURE ENGINEERING COMPLETE ")
print("="*30)
print(f"After basic cleaning shape: {df.shape}")

print("\nMissing values (raw count):")
print(df.isna().sum())

# === ADDED: Missing Value Percentage View ===
print("\nMissing values (percentage):")
missing_pct = (df.isna().sum() / len(df) * 100).sort_values(ascending=False)
print(missing_pct[missing_pct > 0])
# === END ADDED ===

print("\nDataFrame head:")
display(df.head())
# print(df['Delivery_Days'])

In [ ]:

# ==============================================================================
# 📊 START OF EXPLORATORY DATA ANALYSIS (EDA)
# ==============================================================================

print("\n" + "="*30)
print(" STARTING EDA ")
print("="*30)

# 🔹 Define column lists
num_cols = df.select_dtypes(include=['float64', 'int64']).columns.tolist()
# === ADDED: Exclude new date features from 'num_cols' for general stats ===
date_num_features = ['Order_Month', 'Order_Day_of_Week', 'Delivery_Days']
for col in ['Transport_Cost'] + date_num_features:
    if col in num_cols:
        num_cols.remove(col)
# === END ADDED ===
        
cat_cols = df.select_dtypes(include='object').columns.tolist()
# === ADDED: Add boolean 'Is_Weekend' to cat_cols for analysis ===
if 'Order_Is_Weekend' in df.columns:
    cat_cols.append('Order_Is_Weekend')
# === END ADDED ===

print(f"Numeric features identified: {num_cols}")
print(f"Categorical features identified: {cat_cols}")
print(f"Date-derived features identified: {date_num_features}")


# === ADDED: 1. Target Variable Analysis (Transport_Cost) ===
print("\n===== 1. TARGET VARIABLE ANALYSIS: Transport_Cost =====")
plt.figure(figsize=(14, 5))

# Plot 1: Original Distribution
plt.subplot(1, 2, 1)
sns.histplot(df['Transport_Cost'], kde=True, bins=40)
plt.title('Distribution of Transport_Cost (Original)')
plt.xlabel('Transport_Cost')

# Plot 2: Log-Transformed Distribution
# We add 1 to handle potential zero values before logging
plt.subplot(1, 2, 2)
log_target = np.log1p(df['Transport_Cost'])
sns.histplot(log_target, kde=True, bins=40, color='green')
plt.title('Distribution of log(Transport_Cost + 1)')
plt.xlabel('log(Transport_Cost + 1)')

plt.suptitle('Target Variable Distribution Analysis', fontsize=16)
plt.tight_layout(rect=[0, 0.03, 1, 0.95])
plt.show()

print(f"Skewness of Transport_Cost: {df['Transport_Cost'].skew():.4f}")
print(f"Skewness of log(Transport_Cost + 1): {log_target.skew():.4f}")
# === END ADDED ===


print("\n===== 2. NUMERIC FEATURE ANALYSIS =====")
print("===== BASIC NUMERIC STATISTICS =====")
if not num_cols:
    print("No numeric columns found to describe (excluding target/dates).")
else:
    display(df[num_cols].describe().T)

    print("\n===== SKEWNESS =====")
    display(df[num_cols].skew())

# 🔹 Numeric distributions + boxplots
# (Your original loop)
# === MODIFIED: Added a check for empty list ===
print("\nGenerating numeric distribution plots...")
analysis_num_cols = num_cols + ['Delivery_Days'] # Add Delivery_Days back for plotting
if 'Transport_Cost' not in analysis_num_cols:
    analysis_num_cols.append('Transport_Cost') # Add Target back for plotting
    
for col in analysis_num_cols:
    if col in df.columns:
        plt.figure(figsize=(12,4))
        
        plt.subplot(1,2,1)
        sns.histplot(df[col], kde=True, bins=30)
        plt.title(f'{col} distribution')
        
        plt.subplot(1,2,2)
        sns.boxplot(x=df[col])
        plt.title(f'{col} boxplot')
        
        plt.tight_layout()
        plt.show()
    else:
        print(f"Warning: Column '{col}' not found for plotting.")


print("\n===== 3. CORRELATION ANALYSIS =====")
# 🔹 Correlation heatmap
# (Your original code)
plt.figure(figsize=(10,8))
corr = df[num_cols + ['Transport_Cost', 'Delivery_Days']].corr()
sns.heatmap(corr, annot=True, cmap='coolwarm', fmt=".2f")
plt.title('Correlation Heatmap')
plt.show()


print("\n===== 4. CATEGORICAL FEATURE ANALYSIS =====")
# 🔹 Categorical distributions
# (Your original loop)
print("\nGenerating categorical distribution plots...")
high_cardinality_cols = []
for col in cat_cols:
    print(f"\n===== Column: {col} =====")
    print(df[col].value_counts(dropna=False))
    
    nunique = df[col].nunique()
    if nunique > 20:
        high_cardinality_cols.append(col)
        print(f"SKIPPING countplot for {col} (High Cardinality: {nunique} unique values)")
        continue
        
    plt.figure(figsize=(8,4))
    sns.countplot(y=col, data=df, order=df[col].value_counts().index)
    plt.title(f'Count of {col}')
    plt.tight_layout()
    plt.show()

# === ADDED: 4a. High-Cardinality Column Summary ===
print("\n===== 4a. HIGH-CARDINALITY CATEGORICAL SUMMARY =====")
if high_cardinality_cols:
    print(f"High-cardinality features detected: {high_cardinality_cols}")
    for col in high_cardinality_cols:
        print(f"\n--- Top 10 values for: {col} ---")
        print(df[col].value_counts(dropna=False).head(10))
        print(f"...and {df[col].nunique() - 10} other unique values.")
else:
    print("No high-cardinality categorical features detected (threshold > 20).")
# === END ADDED ===


print("\n===== 5. BIVARIATE ANALYSIS (FEATURES vs. TARGET) =====")
# 🔹 Numeric features vs target
# (Your original loop)
print("\nGenerating numeric features vs. Transport_Cost...")
for col in num_cols + ['Delivery_Days']:
    if col in df.columns:
        plt.figure(figsize=(6,4))
        sns.scatterplot(x=df[col], y=df['Transport_Cost'])
        plt.title(f'{col} vs Transport_Cost')
        plt.tight_layout()
        plt.show()

# 🔹 Categorical features vs target (low-cardinality)
# (Your original loop)
print("\nGenerating categorical features vs. Transport_Cost...")
for col in cat_cols:
    if col in df.columns and df[col].nunique() < 20:
        plt.figure(figsize=(10,4))
        sns.boxplot(x=col, y='Transport_Cost', data=df)
        plt.title(f'{col} vs Transport_Cost')
        plt.xticks(rotation=45)
        plt.tight_layout()
        plt.show()

# === ADDED: 5a. Date-Derived Features vs. Target ===
print("\nGenerating date-derived features vs. Transport_Cost...")
date_features_to_plot = ['Order_Month', 'Order_Day_of_Week', 'Order_Is_Weekend']
for col in date_features_to_plot:
    if col in df.columns:
        plt.figure(figsize=(10, 4))
        sns.boxplot(x=col, y='Transport_Cost', data=df)
        plt.title(f'{col} vs Transport_Cost')
        if col == 'Order_Day_of_Week':
            plt.xticks(ticks=range(7), labels=['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun'])
        plt.tight_layout()
        plt.show()
# === END ADDED ===


print("\n===== 6. OUTLIER DETECTION =====")
# 🔹 Outlier detection (Z-score)
# (Your original code)
# === MODIFIED: Added nan_policy='omit' to handle missing values gracefully ===
try:
    z_scores = df[num_cols + ['Transport_Cost', 'Delivery_Days']].apply(lambda x: zscore(x, nan_policy='omit'))
    outliers = (abs(z_scores) > 3).sum()
    print("\n===== NUMBER OF OUTLIERS PER COLUMN (Z-score > 3) =====")
    print(outliers[outliers > 0].sort_values(ascending=False))
except ValueError as e:
    print(f"Could not calculate Z-scores, likely due to all-NaN column. Error: {e}")
# === END MODIFIED ===


print("\n===== 7. MISSING VALUE VISUALIZATION =====")
# 🔹 Missing value visualization
# (Your original code)
print("\nGenerating missing value matrix...")
msno.matrix(df)
plt.title('Missing Value Matrix')
plt.show()

print("\nGenerating missing value bar chart...")
msno.bar(df)
plt.title('Missing Value Bar Chart')
plt.show()


print("\n" + "="*30)
print(" EDA COMPLETE ")
print("="*30)

In [55]:
print("Preprocessing script started...")

# ==============================================================================
# PART 1: PRE-SPLIT (Data Cleaning & Feature Engineering)
# These actions are applied to the whole dataset before splitting.
# ==============================================================================

# 1. Filter Bad Data
# EDA Finding: We found impossible values like Transport_Cost < 0 and Delivery_Days < 0.
# Action: Remove these rows entirely.
initial_rows = len(df)
df = df[df['Transport_Cost'] >= 0]
df = df[df['Delivery_Days'] >= 0]
print(f"Filtered {initial_rows - len(df)} rows with bad data (negative cost or delivery days).")

# 2. Feature Engineering
# EDA Finding: Equipment_Height & Equipment_Width were highly correlated (0.77).
# Action: Combine them into a single 'Equipment_Volume' feature.
df['Equipment_Volume'] = df['Equipment_Height'] * df['Equipment_Width']

# 3. Log-Transform Skewed Features
# EDA Finding: Equipment_Value (skew=24) and our new Equipment_Volume
# (derived from skewed features) are extremely right-skewed.
# Action: Apply np.log1p to normalize them.
df['Equipment_Value'] = np.log1p(df['Equipment_Value'])
df['Equipment_Volume'] = np.log1p(df['Equipment_Volume'])

# 4. Define Target (y) and Features (X)
# EDA Finding: Target 'Transport_Cost' is extremely skewed (skew=30).
# Action: Use np.log1p on the target. We will predict the log, then convert back.
y = np.log1p(df['Transport_Cost']) # this y is in log-space

# Action: Define X by dropping the target, original engineered columns, 
# and high-cardinality/redundant/ID columns identified in the EDA.
X = df.drop(columns=[
    # Target
    'Transport_Cost',
    
    # Replaced by Equipment_Volume
    'Equipment_Height',
    'Equipment_Width',
    
    # Redundant (corr 0.90 with Value)
    'Equipment_Weight',
    
    # High-Cardinality IDs / Unused
    'Hospital_Id',
    'Supplier_Name',
    'Hospital_Location',
    
    # Replaced by date features
    'Order_Placed_Date',
    'Delivery_Date'
])

print(f"Features for modeling: {X.columns.tolist()}")

# 5. Train-Test Split
# Action: Split the data into training and testing sets.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
train_mean = y_train.mean()

# 1️⃣ Baseline RMSE on the training set (log-space)
# y_train_pred_baseline = np.full_like(y_train, train_mean)
# baseline_rmse_train = np.sqrt(mean_squared_error(y_train, y_train_pred_baseline))
# print(f"Baseline RMSE (train, log-space): {baseline_rmse_train:.4f}")

# 2️⃣ Baseline RMSE on the test set (using train mean as predictor) — log-space
y_test_pred_baseline = np.full_like(y_test, train_mean)
baseline_rmse_test_log = np.sqrt(mean_squared_error(y_test, y_test_pred_baseline))
print(f"Baseline RMSE (test, log-space): {baseline_rmse_test_log:.4f}")

# 3️⃣ Baseline RMSE in original (Transport_Cost) scale
y_test_actual_orig = np.expm1(y_test)
y_test_baseline_pred_orig = np.expm1(y_test_pred_baseline)
baseline_rmse_test_orig = np.sqrt(mean_squared_error(y_test_actual_orig, y_test_baseline_pred_orig))
print(f"Baseline RMSE (test, original-scale): {baseline_rmse_test_orig:.4f}")

print(f"Training set shape: {X_train.shape}")
print(f"Test set shape: {X_test.shape}")


# ==============================================================================
# PART 2: POST-SPLIT (Pipelines & ColumnTransformer)
# This prevents data leakage. We FIT on X_train, then TRANSFORM X_train and X_test.
# ==============================================================================

# 1. Define Feature Lists
# Action: Separate our final columns into numeric and categorical lists.

numeric_features = [
    'Supplier_Reliability',
    'Equipment_Value',      # Already log-transformed
    'Base_Transport_Fee',
    'Delivery_Days',
    'Equipment_Volume'      # Already log-transformed
]

categorical_features = [
    'Equipment_Type',
    'CrossBorder_Shipping',
    'Urgent_Shipping',
    'Installation_Service',
    'Transport_Method',
    'Fragile_Equipment',
    'Hospital_Info',
    'Rural_Hospital',
    'Order_Month',
    'Order_Day_of_Week',
    'Order_Is_Weekend'
]

# 2. Create the Numeric Pipeline
# EDA Finding: Numeric features had missing values (e.g., Supplier_Reliability)
# and were on different scales.
# Action: Impute missing values with the median (robust to outliers)
# and then scale all features.
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

# 3. Create the Categorical Pipeline
# EDA Finding: Categorical features had missing values (e.g., Transport_Method,
# Rural_Hospital) and need to be converted to numbers.
# Action: Impute missing values with the most frequent value and then
# one-hot encode. 'handle_unknown='ignore'' ensures our model doesn't
# crash if it sees a new category in the test data.
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
])

# 4. Create the Full Preprocessor
# Action: Combine the numeric and categorical pipelines using ColumnTransformer.
# This single object will handle all preprocessing for us.
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ],
    remainder='drop'  # Drop any columns we didn't explicitly list
)

# 5. Apply the Preprocessor
# Action: FIT the preprocessor on X_train ONLY (to learn medians, modes, etc.)
# and then TRANSFORM both X_train and X_test.
# This gives us our final, model-ready datasets.

print("\nFitting preprocessor on X_train...")
X_train_processed = preprocessor.fit_transform(X_train)
X_test_processed = preprocessor.transform(X_test) # what if we have missig data in testing set?

print("Preprocessing complete.")
print(f"Processed X_train shape: {X_train_processed.shape}")
print(f"Processed X_test shape: {X_test_processed.shape}")


Preprocessing script started...
Filtered 2267 rows with bad data (negative cost or delivery days).
Features for modeling: ['Supplier_Reliability', 'Equipment_Type', 'Equipment_Value', 'Base_Transport_Fee', 'CrossBorder_Shipping', 'Urgent_Shipping', 'Installation_Service', 'Transport_Method', 'Fragile_Equipment', 'Hospital_Info', 'Rural_Hospital', 'Delivery_Days', 'Order_Month', 'Order_Day_of_Week', 'Order_Is_Weekend', 'Equipment_Volume']
Baseline RMSE (test, log-space): 1.7725
Baseline RMSE (test, original-scale): 625946.6068
Training set shape: (2186, 16)
Test set shape: (547, 16)

Fitting preprocessor on X_train...
Preprocessing complete.
Processed X_train shape: (2186, 48)
Processed X_test shape: (547, 48)


In [54]:

# ----- 1) Set up 5-Fold -----
kf = KFold(n_splits=5, shuffle=True, random_state=42)

# ----- 2) Create the pipeline -----
lr_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),  # your ColumnTransformer
    ('model', LinearRegression())
])

# ----- 3) Run 5-Fold Cross-Validation -----
cv_scores = cross_val_score(
    lr_pipeline,
    X_train,       # raw training features
    y_train,       # log-transformed target
    cv=kf,
    scoring='neg_root_mean_squared_error',
    n_jobs=1
)

avg_rmse = np.abs(cv_scores).mean()
print(f"Linear Regression 5-Fold Avg. RMSE (log-space): {avg_rmse:.4f}")

# ----- 4) Fit final model on full training data -----
lr_pipeline.fit(X_train, y_train)
# print("Linear Regression final model trained on full training set.")

# ----- 5) Predict on test set -----
y_test_pred_log = lr_pipeline.predict(X_test)
y_test_pred_orig = np.expm1(y_test_pred_log)  # back to original scale

rmse_test_log = np.sqrt(np.mean((y_test - y_test_pred_log)**2))
rmse_test_orig = np.sqrt(np.mean((np.expm1(y_test) - y_test_pred_orig)**2))

print(f"Test RMSE (log-space)      : {rmse_test_log:.4f}")  
print(f"Test RMSE (original scale) : {rmse_test_orig:.2f}")

Linear Regression 5-Fold Avg. RMSE (log-space): 0.6712
Test RMSE (log-space)      : 0.7249
Test RMSE (original scale) : 622505.75


In [56]:

# ----- 1) Set up 5-Fold -----
kf = KFold(n_splits=5, shuffle=True, random_state=42)

# ----- 2) Create the pipeline -----
poly_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),         # your ColumnTransformer
    ('poly', PolynomialFeatures()),         # will tune degree
    ('model', LinearRegression())
])

# ----- 3) Set up GridSearch for hyperparameter tuning -----
param_grid = {
    'poly__degree': [2,3]   # you can expand to 5 if dataset is small
}

grid_search = GridSearchCV(
    poly_pipeline,
    param_grid,
    scoring='neg_root_mean_squared_error',
    cv=kf,
    n_jobs=1
)

# ----- 4) Run GridSearch -----
grid_search.fit(X_train, y_train)

# Best degree
best_degree = grid_search.best_params_['poly__degree']
best_rmse = -grid_search.best_score_
print(f"Best polynomial degree: {best_degree}")
print(f"Best CV RMSE (log-space): {best_rmse:.4f}")

# ----- 5) Fit final model on full training data -----
final_poly_model = grid_search.best_estimator_
final_poly_model.fit(X_train, y_train)
print("Polynomial Regression final model trained on full training set.")

# ----- 6) Predict on test set -----
y_test_pred_log = final_poly_model.predict(X_test)
y_test_pred_orig = np.expm1(y_test_pred_log)  # back to original scale

rmse_test_log = np.sqrt(np.mean((y_test - y_test_pred_log)**2))
rmse_test_orig = np.sqrt(np.mean((np.expm1(y_test) - y_test_pred_orig)**2))

print(f"Test RMSE (log-space)      : {rmse_test_log:.4f}")
print(f"Test RMSE (original scale) : {rmse_test_orig:.2f}")

Best polynomial degree: 2
Best CV RMSE (log-space): 0.5372
Polynomial Regression final model trained on full training set.
Test RMSE (log-space)      : 0.5201
Test RMSE (original scale) : 482129.09


In [62]:

# ----- 1) Create the pipeline -----
ridge_poly_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),  # your ColumnTransformer
    ('poly', PolynomialFeatures()),  # polynomial expansion
    ('ridge', Ridge())               # ridge regression
])

# ----- 2) Set hyperparameter grid -----
param_grid = {
    'poly__degree': [3],      # try degrees 1, 2, 3, 4, 5
    'ridge__alpha': [ 100]  # try different regularization strengths
}

# ----- 3) Grid Search with 5-Fold CV -----
grid_search = GridSearchCV(
    ridge_poly_pipeline,
    param_grid,
    scoring='neg_root_mean_squared_error',
    cv=5,
    n_jobs=1
)

# ----- 4) Fit on training data -----
grid_search.fit(X_train, y_train)

# ----- 5) Best hyperparameters -----
print("Best hyperparameters:", grid_search.best_params_)
print("Best CV RMSE (log-space):", -grid_search.best_score_)

# ----- 6) Predict on test set -----
y_test_pred_log = grid_search.predict(X_test)
y_test_pred_orig = np.expm1(y_test_pred_log)

rmse_test_log = np.sqrt(np.mean((y_test - y_test_pred_log)**2))
rmse_test_orig = np.sqrt(np.mean((np.expm1(y_test) - y_test_pred_orig)**2))

print(f"Test RMSE (log-space)      : {rmse_test_log:.4f}")
print(f"Test RMSE (original scale) : {rmse_test_orig:.2f}")

Best hyperparameters: {'poly__degree': 3, 'ridge__alpha': 100}
Best CV RMSE (log-space): 0.4981973090176825
Test RMSE (log-space)      : 0.5278
Test RMSE (original scale) : 665702.93


In [64]:

# ----- 1) Create the pipeline -----
lasso_poly_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),  # your ColumnTransformer
    ('poly', PolynomialFeatures()),  # polynomial expansion
    ('lasso', Lasso(max_iter=10000)) # Lasso regression
])

# ----- 2) Set hyperparameter grid -----
param_grid = {
    'poly__degree': [3],       # try degrees 1, 2, 3
    'lasso__alpha': [0.001, 0.01, 0.1, 1, 10]  # regularization strengths
}

# ----- 3) Grid Search with 5-Fold CV -----
grid_search = GridSearchCV(
    lasso_poly_pipeline,
    param_grid,
    scoring='neg_root_mean_squared_error',
    cv=5,
    n_jobs=-1
)

# ----- 4) Fit on training data -----
grid_search.fit(X_train, y_train)

# ----- 5) Best hyperparameters -----
print("Best hyperparameters:", grid_search.best_params_)
print("Best CV RMSE (log-space):", -grid_search.best_score_)

# ----- 6) Predict on test set -----
y_test_pred_log = grid_search.predict(X_test)
y_test_pred_orig = np.expm1(y_test_pred_log)

rmse_test_log = np.sqrt(np.mean((y_test - y_test_pred_log)**2))
rmse_test_orig = np.sqrt(np.mean((np.expm1(y_test) - y_test_pred_orig)**2))

print(f"Test RMSE (log-space)      : {rmse_test_log:.4f}")
print(f"Test RMSE (original scale) : {rmse_test_orig:.2f}")

Best hyperparameters: {'lasso__alpha': 0.01, 'poly__degree': 3}
Best CV RMSE (log-space): 0.441190042720711
Test RMSE (log-space)      : 0.4473
Test RMSE (original scale) : 455988.25


In [66]:

# ----- 1) CV splitter -----
kf = KFold(n_splits=5, shuffle=True, random_state=42)

# ----- 2) Pipeline: preprocessor -> poly -> elastic net -----
enet_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),                       # your ColumnTransformer
    ('poly', PolynomialFeatures(include_bias=False)),     # polynomial expansion (tune degree)
    ('enet', ElasticNet(max_iter=20000, random_state=42)) # ElasticNet regression
])

# ----- 3) Hyperparameter grid -----
param_grid = {
    'poly__degree': [3],                      # try degrees 1..3 (increase carefully)
    'enet__alpha': [0.1, 1, 10,100,1000],   # regularization strengths
    'enet__l1_ratio': [0.01, 0.1,0.2, 0.5, 0.8]               # mix between L1 (1.0) and L2 (0.0)
}

# ----- 4) GridSearchCV (use n_jobs=1 in notebooks to avoid multiprocessing cwd issues) -----
grid_search = GridSearchCV(
    enet_pipeline,
    param_grid,
    scoring='neg_root_mean_squared_error',
    cv=kf,
    n_jobs=1,
    verbose=2
)

# ----- 5) Run grid search -----
print("Starting GridSearchCV for ElasticNet + PolynomialFeatures ...")
grid_search.fit(X_train, y_train)

# ----- 6) Best params & CV score -----
best_params = grid_search.best_params_
best_cv_rmse = -grid_search.best_score_
print(f"\nBest hyperparameters: {best_params}")
print(f"Best CV RMSE (log-space): {best_cv_rmse:.4f}")

# ----- 7) Final model (best estimator) -----
final_enet = grid_search.best_estimator_

# ----- 8) Evaluate on test set -----
y_test_pred_log = final_enet.predict(X_test)
y_test_pred_orig = np.expm1(y_test_pred_log)

rmse_test_log = np.sqrt(mean_squared_error(y_test, y_test_pred_log))
rmse_test_orig = np.sqrt(mean_squared_error(np.expm1(y_test), y_test_pred_orig))

print(f"\nTest RMSE (log-space)      : {rmse_test_log:.4f}")
print(f"Test RMSE (original scale) : {rmse_test_orig:.2f}")

Starting GridSearchCV for ElasticNet + PolynomialFeatures ...
Fitting 5 folds for each of 25 candidates, totalling 125 fits
[CV] END enet__alpha=0.1, enet__l1_ratio=0.01, poly__degree=3; total time=  21.2s
[CV] END enet__alpha=0.1, enet__l1_ratio=0.01, poly__degree=3; total time=  21.9s
[CV] END enet__alpha=0.1, enet__l1_ratio=0.01, poly__degree=3; total time=  23.0s
[CV] END enet__alpha=0.1, enet__l1_ratio=0.01, poly__degree=3; total time=  19.7s
[CV] END enet__alpha=0.1, enet__l1_ratio=0.01, poly__degree=3; total time=  25.5s
[CV] END enet__alpha=0.1, enet__l1_ratio=0.1, poly__degree=3; total time=   5.9s
[CV] END enet__alpha=0.1, enet__l1_ratio=0.1, poly__degree=3; total time=   6.6s
[CV] END enet__alpha=0.1, enet__l1_ratio=0.1, poly__degree=3; total time=   6.7s
[CV] END enet__alpha=0.1, enet__l1_ratio=0.1, poly__degree=3; total time=   6.0s
[CV] END enet__alpha=0.1, enet__l1_ratio=0.1, poly__degree=3; total time=   6.7s
[CV] END enet__alpha=0.1, enet__l1_ratio=0.2, poly__degree=3;

In [67]:

# ----- 1) CV splitter -----
kf = KFold(n_splits=5, shuffle=True, random_state=42)

# ----- 2) XGB pipeline (preprocessor -> xgb) -----
xgb_pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('xgb', XGBRegressor(objective='reg:squarederror',
                         random_state=42,
                         n_jobs=-1,
                         tree_method='hist'))  # 'hist' is faster for larger data
])

# ----- 3) Hyperparameter grid (example) -----
param_grid = {
    'xgb__n_estimators': [100, 300],
    'xgb__max_depth': [3, 6],
    'xgb__learning_rate': [0.01, 0.1],
    'xgb__subsample': [0.8, 1.0]
}

# ----- 4) GridSearchCV -----
grid_search = GridSearchCV(
    estimator=xgb_pipeline,
    param_grid=param_grid,
    scoring='neg_root_mean_squared_error',
    cv=kf,
    n_jobs=1,   # use 1 in notebooks to avoid multiprocessing cwd issues
    verbose=2
)

# ----- 5) Run grid search -----
print("Starting GridSearchCV for XGBoost...")
grid_search.fit(X_train, y_train)

# ----- 6) Best params and CV score -----
best_params = grid_search.best_params_
best_cv_rmse = -grid_search.best_score_
print("\nBest hyperparameters:", best_params)
print(f"Best CV RMSE (log-space): {best_cv_rmse:.4f}")

# ----- 7) Final model (best estimator) -----
final_xgb = grid_search.best_estimator_

# ----- 8) Evaluate on test set -----
y_test_pred_log = final_xgb.predict(X_test)
y_test_pred_orig = np.expm1(y_test_pred_log)

rmse_test_log = np.sqrt(mean_squared_error(y_test, y_test_pred_log))
rmse_test_orig = np.sqrt(mean_squared_error(np.expm1(y_test), y_test_pred_orig))

print(f"\nTest RMSE (log-space)      : {rmse_test_log:.4f}")
print(f"Test RMSE (original scale) : {rmse_test_orig:.2f}")

# ----- 9) (Optional) Feature importances mapped to feature names -----
# This extracts names from the preprocessor (numeric + one-hot cat names)
pre = final_xgb.named_steps['preprocessor']
ohe = pre.named_transformers_['cat'].named_steps['onehot']
num_names = numeric_features
cat_names = list(ohe.get_feature_names_out(categorical_features))
feature_names = np.concatenate([num_names, cat_names])

# xgboost stores feature importances by index (0..n-1)
xgb_model = final_xgb.named_steps['xgb']
importances = xgb_model.feature_importances_

# If shapes mismatch (e.g., due to different handling), ensure lengths match before creating df
if len(importances) == len(feature_names):
    fi_df = pd.DataFrame({
        'feature': feature_names,
        'importance': importances
    }).sort_values('importance', ascending=False)
    print("\nTop 20 XGBoost feature importances:")
    print(fi_df.head(20).to_string(index=False))
else:
    print("\nFeature importance length does not match derived feature name length. Skipping feature-name mapping.")

Starting GridSearchCV for XGBoost...
Fitting 5 folds for each of 16 candidates, totalling 80 fits
[CV] END xgb__learning_rate=0.01, xgb__max_depth=3, xgb__n_estimators=100, xgb__subsample=0.8; total time=   0.1s
[CV] END xgb__learning_rate=0.01, xgb__max_depth=3, xgb__n_estimators=100, xgb__subsample=0.8; total time=   0.1s
[CV] END xgb__learning_rate=0.01, xgb__max_depth=3, xgb__n_estimators=100, xgb__subsample=0.8; total time=   0.1s
[CV] END xgb__learning_rate=0.01, xgb__max_depth=3, xgb__n_estimators=100, xgb__subsample=0.8; total time=   0.1s
[CV] END xgb__learning_rate=0.01, xgb__max_depth=3, xgb__n_estimators=100, xgb__subsample=0.8; total time=   0.1s
[CV] END xgb__learning_rate=0.01, xgb__max_depth=3, xgb__n_estimators=100, xgb__subsample=1.0; total time=   0.1s
[CV] END xgb__learning_rate=0.01, xgb__max_depth=3, xgb__n_estimators=100, xgb__subsample=1.0; total time=   0.1s
[CV] END xgb__learning_rate=0.01, xgb__max_depth=3, xgb__n_estimators=100, xgb__subsample=1.0; total tim

In [ ]:

kf = KFold(n_splits=5, shuffle=True, random_state=42)

lgb_pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('lgb', LGBMRegressor(random_state=42, n_jobs=-1))
])

param_grid = {
    'lgb__n_estimators': [100, 300],
    'lgb__max_depth': [4, 8],
    'lgb__learning_rate': [0.01, 0.1],
    'lgb__num_leaves': [31, 63]
}

grid = GridSearchCV(
    estimator=lgb_pipeline,
    param_grid=param_grid,
    scoring='neg_root_mean_squared_error',
    cv=kf,
    n_jobs=1,     # safer in notebooks; use >1 or -1 in script environments
    verbose=2
)

print("Starting GridSearchCV for LightGBM...")
grid.fit(X_train, y_train)

print("Best params:", grid.best_params_)
best_cv_rmse = -grid.best_score_
print(f"Best CV RMSE (log-space): {best_cv_rmse:.4f}")

# Final model: best estimator already includes the preprocessor
final_lgb = grid.best_estimator_

# Evaluate on test set
y_test_pred_log = final_lgb.predict(X_test)
y_test_pred_orig = np.expm1(y_test_pred_log)

rmse_test_log = np.sqrt(mean_squared_error(y_test, y_test_pred_log))
rmse_test_orig = np.sqrt(mean_squared_error(np.expm1(y_test), y_test_pred_orig))

print(f"Test RMSE (log-space): {rmse_test_log:.4f}")
print(f"Test RMSE (original scale): {rmse_test_orig:.2f}")

# Optional: feature importances mapped to names (if preprocessor produces matching columns)
pre = final_lgb.named_steps['preprocessor']
ohe = pre.named_transformers_['cat'].named_steps['onehot']
num_names = numeric_features
cat_names = list(ohe.get_feature_names_out(categorical_features))
feature_names = np.concatenate([num_names, cat_names])

lgb_model = final_lgb.named_steps['lgb']
importances = lgb_model.feature_importances_

if len(importances) == len(feature_names):
    fi_df = pd.DataFrame({'feature': feature_names, 'importance': importances}) \
             .sort_values('importance', ascending=False)
    print(fi_df.head(20).to_string(index=False))
else:
    print("Warning: feature importance length != feature name length. Skipping mapping.")

In [ ]:
# ----- 1) Pipeline -----
rf_pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('rf', RandomForestRegressor(random_state=42, n_jobs=-1))
])

# ----- 2) Expanded parameter grid -----
param_dist = {
    'rf__n_estimators': [100, 300, 500, 700],
    'rf__max_depth': [None, 10, 20, 30],
    'rf__min_samples_split': [2, 5, 10],
    'rf__min_samples_leaf': [1, 2, 4],
    'rf__max_features': ['sqrt', 'log2', 0.5, 1.0]
}

# ----- 3) 5-Fold CV -----
kf = KFold(n_splits=5, shuffle=True, random_state=42)

# ----- 4) Randomized Search -----
random_search = RandomizedSearchCV(
    rf_pipeline,
    param_distributions=param_dist,
    n_iter=40,
    scoring='neg_root_mean_squared_error',
    cv=kf,
    n_jobs=-1,
    verbose=1,  # shows progress for each combination
    random_state=42
)

# ----- 5) Train -----
print("🚀 Starting RandomizedSearchCV for Random Forest...")
random_search.fit(X_train, y_train)
print("✅ RandomizedSearchCV complete.")

# ----- 6) Evaluate -----
best_model = random_search.best_estimator_
print("🔹 Best model selected. Predicting on test set...")
y_pred_log = best_model.predict(X_test)
y_pred_orig = np.expm1(y_pred_log)

rmse_log = np.sqrt(mean_squared_error(y_test, y_pred_log))
rmse_orig = np.sqrt(mean_squared_error(np.expm1(y_test), y_pred_orig))

# ----- 7) Final Results -----
print("\n===== FINAL RESULTS =====")
print("✅ Best Parameters:", random_search.best_params_)
print(f"✅ CV RMSE (log-space): {-random_search.best_score_:.4f}")
print(f"✅ Test RMSE (log-space): {rmse_log:.4f}")
print(f"✅ Test RMSE (original scale): {rmse_orig:.2f}")

In [ ]:
print("\n" + "="*30)
print(" TRAINING FINAL MODEL ON ALL DATA ")
print("="*30)

print("You've found the best parameters. Now, we'll train a new model using")
print("these parameters on the *entire* dataset (X and y) to create")
print("the final, production-ready model.")

# --- 1. Re-define the unfitted preprocessor ---
# We MUST do this to get a fresh, unfitted preprocessor
# so it can be properly fitted on the *full* X dataset.

# Define Feature Lists (as before)
numeric_features = [
    'Supplier_Reliability', 'Equipment_Value', 'Base_Transport_Fee',
    'Delivery_Days', 'Equipment_Volume'
]
categorical_features = [
    'Equipment_Type', 'CrossBorder_Shipping', 'Urgent_Shipping',
    'Installation_Service', 'Transport_Method', 'Fragile_Equipment',
    'Hospital_Info', 'Rural_Hospital', 'Order_Month',
    'Order_Day_of_Week', 'Order_Is_Weekend'
]

# Create the Numeric Pipeline (unfitted)
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

# Create the Categorical Pipeline (unfitted)
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
])

# Create the Full Preprocessor (unfitted)
final_preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ],
    remainder='drop'
)

# --- 2. Create the final, unfitted XGBoost pipeline ---
# This pipeline contains the unfitted preprocessor and an unfitted model
final_model_pipeline = Pipeline([
    ('preprocessor', final_preprocessor),
    ('xgb', XGBRegressor(objective='reg:squarederror',
                         random_state=42,
                         n_jobs=-1,
                         tree_method='hist'))
])

# --- 3. Get best parameters from your grid search ---
best_params = grid_search.best_params_
print(f"\nUsing best parameters: {best_params}")

# --- 4. Set the best parameters on the new pipeline ---
final_model_pipeline.set_params(**best_params)

# --- 5. Fit the final pipeline on ALL data (X, y) ---
# This will fit the preprocessor (imputers, scalers) on ALL X
# and then train the XGBoost model on ALL X and y.
print("Fitting final model on the entire (X, y) dataset...")
final_model_pipeline.fit(X, y)

print("\nTraining complete!")
print("The 'final_model_pipeline' object is now your fully-trained model,")
print("ready to be saved and used for predictions.")

# --- 6. (Optional) Save your final model ---
# You can now save this model to a file for later use.
# import joblib
# joblib.dump(final_model_pipeline, 'final_xgb_model.pkl')
# print("\nFinal model saved to 'final_xgb_model.pkl'")

In [ ]:

def prepare_features(df_raw):
    """
    Applies all manual cleaning and feature engineering
    to match the data used for model training.
    
    Takes a raw DataFrame (like test.csv) and returns
    a DataFrame ready for the model pipeline.
    """
    # Make a copy to avoid changing the original data
    df = df_raw.copy()
    
    # 1. Clean column names (from your training script)
    df.columns = df.columns.str.strip()

    # 2. Clean all string/object columns (from your training script)
    for col in df.select_dtypes(include='object').columns:
        df[col] = df[col].astype(str).str.strip()
        df[col] = df[col].replace({'': np.nan, 'nan': np.nan, 'NaN': np.nan})

    # 3. Normalize Yes/No columns (from your training script)
    yes_no_cols = ['CrossBorder_Shipping', 'Urgent_Shipping', 'Installation_Service',
                   'Fragile_Equipment', 'Rural_Hospital']
    for col in yes_no_cols:
        if col in df.columns:
            df[col] = df[col].replace({
                'YES': 'Yes', 'yes': 'Yes', 'Y': 'Yes', 'y': 'Yes',
                'NO': 'No', 'no': 'No', 'N': 'No', 'n': 'No'
            })

    # 4. Convert date columns (from your training script)
    df['Order_Placed_Date'] = pd.to_datetime(df['Order_Placed_Date'], errors='coerce')
    df['Delivery_Date'] = pd.to_datetime(df['Delivery_Date'], errors='coerce')

    # 5. Engineer Date Features (from your training script)
    df['Delivery_Days'] = (df['Delivery_Date'] - df['Order_Placed_Date']).dt.days
    df['Delivery_Days'] = pd.to_numeric(df['Delivery_Days'], errors='coerce')
    
    df['Order_Month'] = df['Order_Placed_Date'].dt.month
    df['Order_Day_of_Week'] = df['Order_Placed_Date'].dt.dayofweek
    df['Order_Is_Weekend'] = df['Order_Day_of_Week'].isin([5, 6])
    
    # 6. Handle bad data (CRITICAL FIX)
    # Instead of dropping rows, we set bad data to NaN.
    # Your pipeline's imputer will then handle it.
    df.loc[df['Delivery_Days'] < 0, 'Delivery_Days'] = np.nan
    # let us print how many nans were set
    num_bad_delivery_days = df['Delivery_Days'].isna().sum()
    print(f"Set {num_bad_delivery_days} invalid Delivery_Days to NaN.")

    # 7. Engineer Volume Feature (from your training script)
    df['Equipment_Volume'] = df['Equipment_Height'] * df['Equipment_Width']

    # 8. Log-Transform Skewed Features (from your training script)
    # The model was trained on these log-transformed features.
    df['Equipment_Value'] = np.log1p(df['Equipment_Value'])
    df['Equipment_Volume'] = np.log1p(df['Equipment_Volume'])
    
    # 9. Return the feature-engineered DataFrame
    # The pipeline will select the columns it needs from this.
    return df

In [ ]:
# Assume 'final_model_pipeline' is your trained model object from the previous step
# import joblib
# final_model_pipeline = joblib.load('final_xgb_model.pkl') # If you saved it

# 1. Load your new, raw test data
print("Loading new test data...")
# I'm using 'test.csv' as the example filename
df_new_test = pd.read_csv('../data/test.csv') 

# 2. Save IDs for the final submission
# We need to map our predictions back to the original IDs
submission_ids = df_new_test['Hospital_Id']

# 3. Apply the *exact same* feature engineering
print("Applying feature engineering to new data...")
X_new_prepared = prepare_features(df_new_test)

# 4. Get predictions
# The pipeline will handle the rest:
# - Selects the correct columns
# - Imputes missing values (using 'median'/'most_frequent' from training)
# - Scales numeric features (using 'scaler' from training)
# - One-hot encodes categorical features (using 'onehot' from training)
# - Runs the XGBoost model
print("Getting predictions from the final model...")
log_predictions = final_model_pipeline.predict(X_new_prepared)

# 5. Convert predictions back from log-scale!
# Remember, you trained on log(Transport_Cost + 1)
final_predictions = np.expm1(log_predictions)

# 6. Create the final submission file
submission_df = pd.DataFrame({
    'Hospital_Id': submission_ids,
    'Transport_Cost': final_predictions
})

# Display the first few predictions
print("\nFinal Predictions:")
display(submission_df.head())

# Save to CSV
submission_df.to_csv('submission1.csv', index=False)
print("Submission file 'submission.csv' created successfully.")